<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Package Import </span>

In [210]:
import pandas as pd
from bs4 import BeautifulSoup
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> User Text Box </span>

In [2]:
button_artist = widgets.Button(description="Select")
inp_artist = widgets.Text(description='Artist/Band')
box_get_artist = widgets.VBox([button_artist, inp_artist])

input_artist = []
input_history = []

def on_button_clicked(text_input):
    input_artist.append(inp_artist.value)
    input_history.append(f'{datetime.now()}')
    print(inp_artist.value, 'selected.')
    inp_artist.value = ''    
    
button_artist.on_click(on_button_clicked)


In [235]:
widgets.VBox([box_get_artist])


Metallica selected.
Linkin Park selected.
InvalidArtistName selected.


In [236]:
print(input_artist)
print(input_history)


['Metallica', 'Linkin Park', 'InvalidArtistName']
['2023-09-09 03:58:13.918608', '2023-09-09 03:58:15.671894', '2023-09-09 03:58:37.933679']


<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Search dictionary and store relevant URLs </span>

In [237]:
%run py_utils/generic_utils.ipynb

start_urls = []
root_url = 'https://www.azlyrics.com/'
path = '../datasets/artists_data.json'
all_artists = load_json(path)

for artist in input_artist:
    if artist in all_artists['Artists']:
        start_urls.append(root_url+all_artists['Artists'][artist])
        
    else:
        print(f'{artist} does not exist in the library.')
        input_artist.remove(artist)


InvalidArtistName does not exist in the library.


In [238]:
start_urls

['https://www.azlyrics.com/m/metallica.html',
 'https://www.azlyrics.com/l/linkinpark.html']

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Extract track data from artists </span>

In [239]:
%run crawlers/tracks_crawler.ipynb

tracks_crawler(start_urls)

> Json storage is successful.
> Dataframe storage is successful.
> Dataframe storage is successful.


In [11]:
json_path = '../datasets/discography_data.json'
discography = load_json(json_path)

discography.keys()

dict_keys(['Tracks', 'Artists', 'Albums', 'URLs'])

In [ ]:
###

In [309]:
# WIP - Dummy Code Block
%run py_utils/generic_utils.ipynb
%run py_utils/azlyrics_utils.ipynb

max_cache = 3
output_size =[]
user_artist = []
user_album = []
user_track = []

def get_discography():
    artist_dropdown = widgets.SelectMultiple(
        options=set(discography['Artists']),
        value=[],
        description="Select Artists",
        rows=len(set(discography['Artists'])) 
    )   
    
    album_dropdown = widgets.Dropdown(
        options=[],
        description="Select Album",
    )
    
    track_dropdown = widgets.Dropdown(
        options=[],
        description="Select Tracks",
    )
    
    def populate_album(change):
        selected_artists = change.new
        selected_artists = ''.join(selected_artists)
        
        if selected_artists:
            albums_list = get_albums(selected_artists)
            album_dropdown.options = albums_list
            
            if albums_list:
                album_dropdown.value = albums_list[0]
                
    def populate_songs(change):
        selected_albums = change.new
        selected_albums = ''.join(selected_albums)
        
        if selected_albums:
            tracks_list = get_tracks(selected_albums)
            track_dropdown.options = tracks_list
            
            if tracks_list:
                track_dropdown.value = tracks_list[0]
    
    def get_track(change):
        track_dropdown.value = change.new
        
    def store_user_selection(_):            
        selected_artist = ''.join(artist_dropdown.value)
        selected_album = ''.join(album_dropdown.value) if selected_artist else None
        selected_track = track_dropdown.value
        
        with output: 
            output_size.append(output.msg_id)
            if len(output_size) > max_cache:
                output.clear_output()
                output_size.clear()
                
            if len(user_track) < max_cache:
                user_artist.append(selected_artist)
                user_album.append(selected_album)
                user_track.append(selected_track)
                print('Chosen Track(s): ', selected_track, ' by ', selected_artist, ' from ', selected_album)
                
            else:
                print("Please purchase a gold membership to increase the track cache.")
            
    def reset_selection(_):
        user_artist.clear()
        user_album.clear()
        user_track.clear()
        with output: 
            output.clear_output()
            print("Cache reset.")     
    
    artist_dropdown.observe(populate_album, names='value')   
    album_dropdown.observe(populate_songs, names='value') 
    track_dropdown.observe(get_track, names='value')    
    
    output = widgets.Output()
    button_track = widgets.Button(description="Store Track") 
    button_track.on_click(store_user_selection)   
    
    
    button_reset = widgets.Button(description="Reset") 
    button_reset.on_click(reset_selection)   
    
    print('Note: Free Membership only allows up to 3 songs.\n')
    artist_selection = widgets.VBox([artist_dropdown, album_dropdown, track_dropdown, 
                                     widgets.HBox([button_track, button_reset]), output])
    display(artist_selection) 
    

In [310]:
get_discography()

Note: Free Membership only allows up to 3 songs.



In [312]:
user_track


['High Voltage', 'Step Up', 'Frantic']

In [ ]:
##

<span style="font-family:Lucida Sans Unicode; color:#a10a0a; font-size: 25px"> Extract lyrics data from tracks </span>

In [5]:
%run crawlers/lyrics_crawler.ipynb

start_urls = ['https://www.azlyrics.com/lyrics/beatles/yellowsubmarine.html', 
              'https://www.azlyrics.com/lyrics/linkinpark/keystothekingdom.html', 
              'https://www.azlyrics.com/lyrics/beatles/1822.html']

lyrics_crawler(start_urls)

3 3 3 3 3
> Json storage is successful.
> Dataframe storage is successful.
> Dataframe storage is successful.
